## Load MODFLOW 6 Model with Flopy

The purpose of this notebook is to demonstrate the how to load an exisiting model using Flopy 

### Setup the Notebook Environment

In [6]:
import os

In [ ]:
import sys
from tempfile import TemporaryDirectory

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import flopy

print(sys.version)
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))
print("flopy version: {}".format(flopy.__version__))

In [8]:
# Model input files and output files will reside here.

sim_workspace =  os.path.join(".", "mf6Well")

### Load the Flopy Model Objects



In [9]:
model_name = "well_00"


In [ ]:
# Load the Flopy simulation object
sim = flopy.mf6.MFSimulation.load(
    sim_ws=sim_workspace, exe_name="mf6", verbosity_level=0)
print(sim.gwf)

### Post-Process head file and cell budget file outputs

Post-processing MODFLOW 6 results is still a work in progress.  There aren't any Flopy plotting functions built in yet, like they are for other MODFLOW versions.  So we need to plot the results using general Flopy capabilities.  We can also use some of the Flopy ModelMap capabilities for MODFLOW 6, but in order to do so, we need to manually create a SpatialReference object, that is needed for the plotting.  Examples of both approaches are shown below.

First, a link to the heads file is created with `HeadFile`. The link can then be accessed with the `get_data` function, by specifying, in this case, the step number and period number for which we want to retrieve data. A three-dimensional array is returned of size `nlay, nrow, ncol`. Matplotlib contouring functions are used to make contours of the layers or a cross-section.

In [ ]:
# Extract the GWF model from the simulation
ml = sim.get_model("gwf_1")


The head file output and cell budget file output can be loaded from either the model object or the OC package object.

In [ ]:
bud = ml.output.budget()
bud.get_data(idx=0, full3D=True)

In [ ]:
hds = ml.output.head()
hds.get_data()

In [ ]:
bud = ml.oc.output.budget()
bud.list_records()

bud.get_data(text='WEL')

In [ ]:
# Extract the CHD flows


In [ ]:
hds = ml.oc.output.head()
h = hds.get_data()
h.shape

In [29]:
grid = ml.modelgrid
grid


In [ ]:
# extract cell center coordinates for plot
x0 = grid.get_xcellcenters_for_layer(layer=0)
y0 = grid.get_ycellcenters_for_layer(layer=0)

In [ ]:

min_hd = round(np.min(h[:, :, :]),0)
c = plt.contour(x0, y0, h[0,:, :], np.arange(min_hd, 0, 0.5))
plt.clabel(c, fmt="%1.1f")
plt.axis("scaled")

In [ ]:
# We can also use the Flopy PlotMapView capabilities for MODFLOW 6
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
modelmap = flopy.plot.PlotMapView(model=ml, layer=0, ax=ax)

# Then we can use the plot_grid() method to draw the grid
# The return value for this function is a matplotlib LineCollection object,
# which could be manipulated (or used) later if necessary.
quadmesh = modelmap.plot_bc('CHD')
ww = modelmap.plot_bc('WEL')
linecollection = modelmap.plot_grid()
contours = modelmap.contour_array(h[-1], levels=np.arange(min_hd, 0, 0.5))

In [ ]:
# We can also use the Flopy PlotMapView capabilities for MODFLOW 6
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

# Next we create an instance of the ModelMap class
modelmap = flopy.plot.PlotMapView(model=ml, ax=ax)

# Then we can use the plot_grid() method to draw the grid
# The return value for this function is a matplotlib LineCollection object,
# which could be manipulated (or used) later if necessary.
linecollection = modelmap.plot_grid()
pa = modelmap.plot_array(h[0])
cb = plt.colorbar(pa, shrink=0.5)

### Post-Process Flows

MODFLOW 6 writes a binary grid file, which contains information about the model grid.  MODFLOW 6 also writes a binary budget file, which contains flow information.  Both of these files can be read using Flopy capabilities.  The MfGrdFile class in Flopy can be used to read the binary grid file.  The CellBudgetFile class in Flopy can be used to read the binary budget file written by MODFLOW 6.

In [ ]:
# read the binary grid file
fname = os.path.join(sim_workspace, "{}.dis.grb".format('gwf_well_00'))
bgf = flopy.mf6.utils.MfGrdFile(fname)

# data read from the binary grid file is stored in a dictionary
bgf._datadict

### Determine the Flow Residual

The FLOW-JA-FACE cell-by-cell budget data can be processed to determine the flow residual for each cell in a MODFLOW 6 model. The diagonal position for each row in the FLOW-JA-FACE cell-by-cell budget data contains the flow residual for each cell and can be extracted using the flopy.mf6.utils.get_residuals() function.

In [50]:
# read the cell budget file
flowja = bud.get_data(text="FLOW-JA-FACE")[0][0, 0, :]
chdflow = bud.get_data(text="CHD")[0]

In [61]:
residual = flopy.mf6.utils.get_residuals(flowja, grb_file=fname)

# Plot a Map of the flow error in Layer 0 (Top Layer)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6), constrained_layout=True)
ax.set_title("Model Layer 0")
modelmap = flopy.plot.PlotMapView(model=ml, ax=ax, layer=0)
pa = modelmap.plot_array(residual)
quadmesh = modelmap.plot_bc("CHD")
linecollection = modelmap.plot_grid(lw=0.5, color="0.5")
contours = modelmap.contour_array(
    h,
    levels=np.arange(min_hd, 0, 0.5),
    colors="black",
)
ax.clabel(contours, fmt="%2.1f")
plt.colorbar(pa, shrink=0.5);

